
# This project aims to find a balance between edge and cloud nodes




Here are all the necessary libraries imported. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from joblib import Parallel, delayed

The Inverse Reinforcement Learning search algorithm gives insights into the weights' distribution and the number of nodes it converges. In the initial example "./300_scores_benchmark.json" including data from 300 machines and six model types, the maximum possible convergence for edge nodes is 300, which means 0 cloud nodes and the minimum convergence of edge nodes is 0, which means 300 cloud nodes.

The method parallel_random_init() starts the search with random values in parallel and is used to find a set of balanced weights. The reward_function() returns the converged environment (as the number of edge nodes) and the received reward.

In [ ]:
class IRL_Search:
    def __init__(self, data):
        self.data = data.copy()
        self.results = None
        self.num_models = np.count_nonzero(self.data["model"].unique())

    def reward_function(self, weights):
        df_data = self.data.copy()  
        df_result = pd.DataFrame()     
        df_data["reward"] = (1 - df_data["cpu_cost_normalized"]) * weights[0]  + (1 - df_data["mse_normalized"]) * 0.5 * weights[1] + (1 - df_data["mae_normalized"]) * 0.5 * weights[1] + (1 - (df_data["latency_normalized"])) * weights[2]
        df_result["machine"] = df_data.groupby("machine").apply(lambda x: x["machine"].iloc[0])
        df_result["instance"] = df_data.groupby("machine").apply(lambda x: np.argmax(x["reward"], axis=0))
        return np.count_nonzero((df_result["instance"].to_numpy() + 1) > self.num_models), df_data.groupby("machine").apply(lambda x: np.max(x["reward"], axis=0)).sum()

    def parallel_random_init(self, n_iterations, n_jobs=-1):
        samples = [self.generate_random_sample() for _ in range(n_iterations)]
        results = Parallel(n_jobs=n_jobs)(delayed(self.evaluate_sample)(s) for s in samples)
        self.results = results

    def get_results(self):
        list = []
        for edge_count, reward, solution in self.results:
             list.append([edge_count, reward, solution])
        return pd.DataFrame(list)
    
    def evaluate_sample(self, weights):
        edge_count, reward = self.reward_function(weights)
        return edge_count, reward, weights

    def generate_random_sample(self):
        x1, x2, x3 = np.random.dirichlet(np.ones(3), size=1)[0]
        return [x1, x2, x3]



Here is the data loaded.

In [ ]:
irl_search300 = IRL_Search(pd.DataFrame.from_dict(json.load(open("./300_scores_benchmark.json"))))
irl_searchoversample = IRL_Search(pd.DataFrame.from_dict(json.load(open("./2700_oversample_scores_benchmark.json"))))

The search is in this part initialized and the rewards are minimized and maximized.

In [ ]:
irl_search300.parallel_random_init(500)
irl_result300 = irl_search300.get_results()

reward_maximized = irl_result300.loc[irl_result300.groupby(0)[1].idxmax()].reset_index().to_numpy()
reward_minimized = irl_result300.loc[irl_result300.groupby(0)[1].idxmin()].reset_index().to_numpy()


Visualization of maximum reward.

In [ ]:
x = []
y1 = []
y2 = []
y3 = []

for r in reward_maximized:
    x.append(r[1])
    y1.append(r[3][0])
    y2.append(r[3][1])
    y3.append(r[3][2])

y_stacked = np.vstack((y1, y2, y3))

fig, ax = plt.subplots(figsize=(15, 6),  dpi=300)
ax.bar(x, y_stacked[0], label='cpu (w1)', color="#546671")
ax.bar(x, y_stacked[1], bottom=y_stacked[0], label='mae_mse (w2)', color="#acb8bb")
ax.bar(x, y_stacked[2], bottom=y_stacked[0] + y_stacked[1], label='latency (w3)', color="#d5d2cc")
ax.set_xlabel("number of edge nodes: cloud ←→ edge")
ax.set_ylabel('weights')
ax.set_title('Maximum Reward')
fig.legend(loc="upper center", bbox_to_anchor=(0.5, -0.001), ncol=len(y_stacked))

Visualization of minimum reward.

In [ ]:
x = []
y1 = []
y2 = []
y3 = []

for r in reward_minimized:
    x.append(r[1])
    y1.append(r[3][0])
    y2.append(r[3][1])
    y3.append(r[3][2])

y_stacked = np.vstack((y1, y2, y3))

fig, ax = plt.subplots(figsize=(15, 6),  dpi=300)
ax.bar(x, y_stacked[0], label='cpu (w1)', color="#546671")
ax.bar(x, y_stacked[1], bottom=y_stacked[0], label='mae_mse (w2)', color="#acb8bb")
ax.bar(x, y_stacked[2], bottom=y_stacked[0] + y_stacked[1], label='latency (w3)', color="#d5d2cc")
ax.set_xlabel("number of edge nodes: cloud ←→ edge")
ax.set_ylabel('weights')
ax.set_title('Minimum Reward')
fig.legend(loc="upper center", bbox_to_anchor=(0.5, -0.001), ncol=len(y_stacked))

In [ ]:
x = []
y1 = []
y2 = []
x_latency = []
y1_latency = []
y2_latency = []
for i in np.concatenate([np.arange(0.0, 0.01, 0.001), np.arange(0.01, 1.01, 0.01)]):
    weights = [np.round(i, 3), np.round(1.0 - i, 3), 0.0]
    result = irl_search300.reward_function(weights)
    x.append(weights[0])
    y1.append(result[0])
    y2.append(result[1])

    weights = [np.round(i, 3), np.round(1.0 - i, 3), 0.2]
    result = irl_search300.reward_function(weights)
    x_latency.append(weights[0])
    y1_latency.append(result[0])
    y2_latency.append(result[1])


Visualization of latency weight 0.0 and 0.2 for 300 nodes

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(13, 3.7),  dpi=300)
ax[0].set_title('Edge nodes involving latency weight: Comparison')
ax[0].plot(x_latency, y1_latency, label='latency weight (w3): 0.20', color="#546671")
ax[0].set_ylim([0,380])
ax[0].plot(x, y1, label='latency weight (w3): 0.00', color="#acb8bb")

ax[0].set_xlabel("weights: cpu (w1) and dependent mae_mse (w2), w2 = (1 - w1)")
ax[0].set_ylabel('number of edge nodes')
ax[0].axvline(x = 0.04, color = 'black', linestyle = ':', label = "0.04")
ax[0].legend(loc='lower right')

ax[1].set_title('Reward involving latency weight: Comparison')
ax[1].set_ylim([0,380])
ax[1].plot(x_latency, y2_latency, label='latency weight (w3): 0.20', color="#546671")
ax[1].plot(x, y2, label='latency weight (w3): 0.00', color="#acb8bb")

ax[1].set_xlabel("weights: cpu (w1) and dependent mae_mse (w2), w2 = (1 - w1)")
ax[1].set_ylabel('reward')
ax[1].legend(loc='lower right')

In [ ]:
x = []
y1 = []
y2 = []
x_latency = []
y1_latency = []
y2_latency = []
for i in np.concatenate([np.arange(0.0, 0.01, 0.001), np.arange(0.01, 1.01, 0.01)]):
    weights = [np.round(i, 3), np.round(1.0 - i, 3), 0.0]
    result = irl_searchoversample.reward_function(weights)
    x.append(weights[0])
    y1.append(result[0])
    y2.append(result[1])

    weights = [np.round(i, 3), np.round(1.0 - i, 3), 0.2]
    example = irl_searchoversample.reward_function(weights)
    x_latency.append(weights[0])
    y1_latency.append(result[0])
    y2_latency.append(result[1])


Visualization of latency weight 0.0 and 0.2 for oversampled nodes.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(13, 3.7),  dpi=300)
ax[0].set_title('Edge nodes involving latency weight: Comparison')
ax[0].plot(x_latency, y1_latency, label='latency weight (w3): 0.20', color="#546671")
ax[0].set_ylim([0,3800])
ax[0].plot(x, y1, label='latency weight (w3): 0.00', color="#acb8bb")

ax[0].set_xlabel("weights: cpu (w1) and dependent mae_mse (w2), w2 = (1 - w1)")
ax[0].set_ylabel('number of edge nodes')
ax[0].axvline(x = 0.48, color = 'black', linestyle = ':', label = "0.48")
ax[0].legend(loc='lower right')

ax[1].set_title('Reward involving latency weight: Comparison')
ax[1].set_ylim([0,3800])
ax[1].plot(x_latency, y2_latency, label='latency weight (w3): 0.20', color="#546671")
ax[1].plot(x, y2, label='latency weight (w3): 0.00', color="#acb8bb")

ax[1].set_xlabel("weights: cpu (w1) and dependent mae_mse (w2), w2 = (1 - w1)")
ax[1].set_ylabel('reward')
ax[1].legend(loc='lower right')

MIT License

Copyright (c) 2023 Björn Leickert

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.